In [33]:
from configparser import ConfigParser
from elasticsearch import Elasticsearch
import re
import os
from dotenv import load_dotenv
import csv
from elasticsearch.helpers import streaming_bulk
from tqdm.notebook import tqdm
import pandas as pd


In [10]:
load_dotenv()

kibana_url = os.getenv('KIBANA_URL')

auth = re.search('https\:\/\/(.*)\@', kibana_url).group(1).split(':')
host = kibana_url.replace('https://%s:%s@' % (auth[0], auth[1]), '')

match = re.search('(:\d+)', host)
if match:
    p = match.group(0)
    host = host.replace(p, '')
    port = int(p.split(':')[1])
else:
    port = 443

es_header = [{
    'host': host,
    'port': port,
    'use_ssl': True,
    'http_auth': (auth[0],auth[1])
}]

es = Elasticsearch(es_header)
es.ping()

True

Bulk add:
https://github.com/elastic/elasticsearch-py/blob/main/examples/bulk-ingest/bulk-ingest.py

In [34]:
# Open file
def generate_actions():
    with open('../Data/wiki.csv', encoding="utf-8") as file_obj:

        # Create reader object by passing the file 
        # object to reader method
        reader_obj = csv.reader(file_obj)

        # Iterate over each row in the csv 
        # file using reader object
        for row in reader_obj:
            doc = {
                'title': row[1],
                'href': row[0]
            }
            yield doc

In [36]:
# read CSV file
number_of_docs = 10704

progress = tqdm(unit="docs", total=number_of_docs)
successes = 0
for ok, action in streaming_bulk(
    client=es, index="wiki", actions=generate_actions(),
):
    progress.update(1)
    successes += ok
    
print("Indexed %d/%d documents" % (successes, number_of_docs))

  0%|          | 0/10704 [00:00<?, ?docs/s]

100%|█████████████████████████████████████████████████████████████████████████▉| 10703/10704 [09:05<00:00, 19.61docs/s]


Indexed 10703/10704 documents
